In [1]:
import logging
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import math
import pandas as pd
import numpy as np
import argparse
import json
import gc
import re
import copy
import random
import datetime
nowdate = datetime.datetime.now()
from tqdm import tqdm
from IPython import get_ipython

from dataclasses import dataclass, field
from datasets import load_dataset
from datasets import load_metric
from typing import Dict, List, Optional, Tuple

from Library.bart_to_long import *
from Library.data_preprocess import *

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from transformers import PreTrainedTokenizerFast, LEDForConditionalGeneration, AutoModel
from transformers import BartForConditionalGeneration, BartConfig
from transformers.models.bart.modeling_bart import BartLearnedPositionalEmbedding
from transformers.models.longformer.modeling_longformer import LongformerSelfAttention
from transformers import get_linear_schedule_with_warmup, AdamW, TrainingArguments

from torchinfo import summary
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import CrossEntropyLoss, MSELoss
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset

import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, LearningRateMonitor
from pytorch_lightning import *
from pytorch_lightning.core.lightning import LightningModule
from torch.utils.data import random_split
from pytorch_lightning import loggers as pl_loggers
from transformers.optimization import AdamW, get_cosine_schedule_with_warmup
import pytorch_lightning.metrics.functional as FM
from pytorch_lightning.loggers import WandbLogger, TensorBoardLogger, MLFlowLogger

import mlflow
from pyngrok import ngrok
from mlflow.tracking import MlflowClient

In [3]:
def sampling_func(data, sample_pct):
    np.random.seed(123)
    N = len(data)
    sample_n = int(len(data)*sample_pct) # integer    
    sample = data.take(np.random.permutation(N)[:sample_n])
    return sample

# context 토큰 길이 
def token_len(text):
    return len(tokenizer.tokenize(text))

In [4]:
def load_data(path):
    tar_train = pd.read_csv(path, encoding='utf-8', index_col = False)
    train_data, val_data = train_test_split(tar_train, test_size=0.3, shuffle=True, stratify=tar_train['Subject'], random_state=42)
    return train_data.to_csv('/Users/yechansmacbook/RS/portfolio/study/nlp/LED_KoBART/data/new_train_data.csv', encoding='utf-8', index=False), val_data.to_csv('/Users/yechansmacbook/RS/portfolio/study/nlp/LED_KoBART/data/new_val_data.csv', encoding='utf-8', index=False)


def sample_set(path):
    tar_train = pd.read_csv(path, encoding='utf-8', index_col = False)
    sample_set = tar_train.groupby('Subject',group_keys=False).apply(sampling_func, sample_pct = 0.2)
    sample_set.reset_index(inplace=True)
    del sample_set['index']
    sample_set['con_token_len'] = sample_set['Content'].apply(token_len)
    tar_train = sample_set[sample_set['con_token_len']<=4096]
    del tar_train['con_token_len']
    train_data, val_data = train_test_split(tar_train, test_size=0.3, shuffle=True, stratify=tar_train['Subject'], random_state=42)
    return train_data.to_csv('/Users/yechansmacbook/RS/portfolio/study/nlp/LED_KoBART/data/train_data_new.csv', encoding='utf-8', index=False), val_data.to_csv('/Users/yechansmacbook/RS/portfolio/study/nlp/LED_KoBART/data/val_data_new.csv', encoding='utf-8', index=False)


In [5]:
load_data('/Users/yechansmacbook/RS/portfolio/study/nlp/LED_KoBART/data/train_data.csv')

(None, None)

In [6]:
sample_set('/Users/yechansmacbook/RS/portfolio/study/nlp/LED_KoBART/data/new_train_data.csv')

(None, None)